## PREDICTION MOVING

In [3]:
#http://stackoverflow.com/questions/11845055/predicting-from-previous-datevalue-data
import matplotlib.pyplot as plt
import pandas as pd # v. 1.7
import MySQLdb
import numpy as np
import time
import sys
import datetime 
from datetime import datetime
from sympy import *
from sympy.solvers import solve
from sympy.parsing.sympy_parser import parse_expr
from sklearn.metrics import mean_absolute_error,mean_squared_error
import statistics 

import requests
%matplotlib inline

import MySQLdb
def initial_trend(series, slen):
    sum = 0.0
    for i in range(slen):
        sum += float(series[i+slen] - series[i]) / slen
    return sum / slen

# >>> initial_trend(series, 12)
# -0.7847222222222222

def triple_exponential_smoothing(series, slen, alpha, beta, gamma, n_preds):
    result = []
    seasonals = initial_seasonal_components(series, slen)
    for i in range(len(series)+n_preds):
        if i == 0: # initial values
            smooth = series[0]
            trend = initial_trend(series, slen)
            result.append(series[0])
            continue
        if i >= len(series): # we are forecasting
            m = i - len(series) + 1
            result.append((smooth + m*trend) + seasonals[i%slen])
        else:
            val = series[i]
            last_smooth, smooth = smooth, alpha*(val-seasonals[i%slen]) + (1-alpha)*(smooth+trend)
            trend = beta * (smooth-last_smooth) + (1-beta)*trend
            seasonals[i%slen] = gamma*(val-smooth) + (1-gamma)*seasonals[i%slen]
            result.append(smooth+trend+seasonals[i%slen])
    return result

# # forecast 24 points (i.e. two seasons)
# >>> triple_exponential_smoothing(series, 12, 0.716, 0.029, 0.993, 24)
# [30, 20.34449316666667, 28.410051892109554, 30.438122252647577, 39.466817731253066, ...


def initial_seasonal_components(series, slen):
    seasonals = {}
    season_averages = []
    n_seasons = int(len(series)/slen)
    # compute season averages
    for j in range(n_seasons):
        season_averages.append(sum(series[slen*j:slen*j+slen])/float(slen))
    # compute initial values
    for i in range(slen):
        sum_of_vals_over_avg = 0.0
        for j in range(n_seasons):
            sum_of_vals_over_avg += series[slen*j+i]-season_averages[j]
        seasonals[i] = sum_of_vals_over_avg/n_seasons
    return seasonals
##-----------------------------------------------------------------------------------
from sklearn.metrics import mean_squared_error
def fittingparameter(data_y,j):
    count_alpha = 0
    count_beta = 0
    count_gamma = 0
    alpha = float(0)
    beta = float(0)
    gamma = float(0)
    index_alpha =[]
    index_beta =[]
    index_gamma = []
    act = j
    data = data_y
    pred_no = 1
    ramda = 0.1
    MSE_holtload = []
#     query1 = "SELECT active_power FROM prediction.data15min_4y WHERE Timestamp >= '%s' AND TimeStamp <= '%s';"% (time_b[act+1],time_b[act+96])
    query1 = "SELECT total_watt FROM thesis.load WHERE Timestamp = '%s' ;"% (time_b_last[act])
    print query1
    df1 = pd.read_sql(query1, conn)
    while (alpha <= float(1)):
        count_alpha += 1
        beta = float(0)
        while (beta <=float(1)):
            count_beta += 1 
            gamma = float(0)
            while (gamma <= float(1)):
                count_gamma += 1
                pred_h = triple_exponential_smoothing(data, 96*7,alpha,beta ,gamma, pred_no)
                pred_data = pred_h[len(pred_h)-(pred_no):]                                               
                MSE = mean_squared_error (df1.total_watt, pred_data)   
#                 print MSE              
                MSE_holtload.append(MSE)
                index_alpha.append(alpha)
                index_beta.append(beta)
                index_gamma.append(gamma)     
#                 print index_alpha,index_beta,index_gamma
                gamma += ramda
            beta += ramda
        alpha += ramda  
#     print  count_gamma,count_beta,count_alpha    
    index_MSE = MSE_holtload.index(min(MSE_holtload))
#     print min(MSE_holtload)
#     print index_MSE
    MSE_min = MSE_holtload[index_MSE]
    alpha = index_alpha[index_MSE]
    beta = index_beta[index_MSE]
    gamma= index_gamma[index_MSE]
#     print count_alpha,count_beta,count_gamma
    print MSE_min,alpha,beta,gamma
    return (alpha,beta,gamma)

def notify(msg):
    URL = "https://notify-api.line.me/api/notify"
    ACCESS_TOKEN = "vxm278yAqFQcXbo2DlTbLD7pqO0QWU2McdNbiwhJWwp"
    MESSAGE_FIELD = {"message" : msg}
    LINE_HEADERS = { "Authorization": "Bearer " + ACCESS_TOKEN }
    try:
        response = requests.post( url=URL,headers=LINE_HEADERS,data=MESSAGE_FIELD)
#         print("Response HTTP Status Code: {status_code}".format( status_code=response.status_code))
    except requests.exceptions.RequestException:
        print("HTTP Request failed")

In [4]:
conn = MySQLdb.connect("localhost","conn","hems")
c = conn.cursor()
query = "SELECT Timestamp, total_watt FROM thesis.load WHERE Timestamp >= '%s'   AND TimeStamp <= '%s';"% ("2016-09-01 00:14:59","2016-11-30 23:59:59")
df = pd.read_sql(query, conn , index_col=['Timestamp'])

pred_no = 1 # power 1 Data/15min in 1Hr --> 60/15 = 4 and 1day = 4*24 = 96
j = 0
i = 0
d = 31
day =0

conn = MySQLdb.connect("localhost","conn","hems")
c = conn.cursor()

time_a = pd.date_range('2016-10-01 00:00:00', periods=96*d, freq='15T')
time_b = pd.date_range('2016-10-01 00:14:59', periods=96*d, freq='15T')

time_a_last = pd.date_range('2016-09-01 00:14:59', periods=96*d, freq='15T')
time_b_last = pd.date_range('2016-09-30 23:59:59', periods=(96*d)+1, freq='15T')

# set databases to empty 
query_empty = """truncate table thesis.predict_moving1""" 
c.execute(query_empty)

while (j < 96*d ) :    
    if (j%96 == 0):    
        day += 1
        print "________DAY:" , day
    
    print "\ndata at :" , j,">>>", time_b[j]
    query_last = "SELECT Timestamp,total_watt FROM  thesis.load WHERE Timestamp >= '%s'   AND TimeStamp <= '%s';"% (time_a_last[j],time_b_last[j])
    print query_last
    df = pd.read_sql(query_last, conn)
    conn.commit()
    data_y = df['total_watt'].values.tolist()
    ans_fiting = fittingparameter(data_y,j)
    
    query_last_new = "SELECT Timestamp,total_watt FROM  thesis.load WHERE Timestamp>='%s'AND Timestamp<='%s';"%(time_a_last[j],time_b_last[j])
    query_last_new_1 = "SELECT Timestamp,total_watt FROM  thesis.load  WHERE Timestamp = '%s' ;"% (time_b_last[j+1])
    print "\n",query_last_new  
    print query_last_new_1
    print "BY USE alpha =",ans_fiting[0],"beta =",ans_fiting[1],"gamma =",ans_fiting[2]
    df_last_new = pd.read_sql(query_last_new, conn)
    conn.commit()
    
    data_y = df_last_new['total_watt'].values.tolist()     
    pred_h = triple_exponential_smoothing(data_y, 96*7, ans_fiting[0], ans_fiting[1], ans_fiting[2],1)
    pred_data = pred_h[len(pred_h)-(pred_no):]   
    pred_watt = abs(pred_data[0])

    c.execute("INSERT INTO thesis.predict_moving1 (Timestamp, watt) VALUES ( %s, %s)",(time_b[j],abs(pred_data[0])))
    conn.commit()  
    
    if ( j == 2880):
        notify("finish moving prediction") 
    j += 1 
print "DONE!!"



________DAY: 1

data at : 0 >>> 2016-10-01 00:14:59
SELECT Timestamp,total_watt FROM  thesis.load WHERE Timestamp >= '2016-09-01 00:14:59'   AND TimeStamp <= '2016-09-30 23:59:59';
SELECT total_watt FROM thesis.load WHERE Timestamp = '2016-09-30 23:59:59' ;
0.0231585663847 0.7 0.1 0.4

SELECT Timestamp,total_watt FROM  thesis.load WHERE Timestamp>='2016-09-01 00:14:59'AND Timestamp<='2016-09-30 23:59:59';
SELECT Timestamp,total_watt FROM  thesis.load  WHERE Timestamp = '2016-10-01 00:14:59' ;
BY USE alpha = 0.7 beta = 0.1 gamma = 0.4

data at : 1 >>> 2016-10-01 00:29:59
SELECT Timestamp,total_watt FROM  thesis.load WHERE Timestamp >= '2016-09-01 00:29:59'   AND TimeStamp <= '2016-10-01 00:14:59';
SELECT total_watt FROM thesis.load WHERE Timestamp = '2016-10-01 00:14:59' ;
0.000227996331065 0.8 0.2 0.0

SELECT Timestamp,total_watt FROM  thesis.load WHERE Timestamp>='2016-09-01 00:29:59'AND Timestamp<='2016-10-01 00:14:59';
SELECT Timestamp,total_watt FROM  thesis.load  WHERE Timestamp =